In [5]:
import tensorflow as tf
from tensorflow.keras import Sequential,mixed_precision
from tensorflow.keras.losses import CategoricalCrossentropy
from tensorflow.keras.layers import InputLayer,Dropout, Dense, Conv2D, MaxPooling2D, Flatten
from tensorflow.keras.layers import RandomFlip,RandomRotation,RandomZoom,RandomBrightness
from tensorflow.keras.optimizers import Adam
from tensorflow.keras.regularizers import l2
from tensorflow.keras.callbacks import EarlyStopping, ReduceLROnPlateau
import matplotlib.pyplot as plt
import numpy as np
from datetime import datetime
import os

os.chdir('/tf-acno-projects/image-classification/')
os.environ['TF_XLA_FLAGS'] = '--tf_xla_enable_xla_devices'

In [6]:
tf.config.optimizer.set_jit(True)  # Enable XLA
gpus = tf.config.experimental.list_physical_devices('GPU')
if gpus:
    try:
        for gpu in gpus:
            tf.config.experimental.set_virtual_device_configuration(
                gpu,
                [tf.config.experimental.VirtualDeviceConfiguration(memory_limit=6000)]  # Limit GPU memory usage
            )
            print(f"Memory limit set for GPU")
    except RuntimeError as e:
        print(e)
        
# Before training
mixed_precision.set_global_policy('mixed_float16')
print("\nXLA Status Check:")
print(f"XLA JIT enabled: {tf.config.optimizer.get_jit()}")
print(f"XLA devices: {tf.config.list_logical_devices('XLA_GPU')}")

Memory limit set for GPU

XLA Status Check:
XLA JIT enabled: autoclustering
XLA devices: [LogicalDevice(name='/device:XLA_GPU:0', device_type='XLA_GPU')]


In [7]:
AUTOTUNE = tf.data.AUTOTUNE
data_dir = 'faces_data/'
input_shape=(128,128,3)

In [4]:
data_augmentation = Sequential([
    RandomRotation(0.1),
    RandomZoom(0.1),
    RandomFlip("horizontal"),
    RandomBrightness(0.1),
])

In [5]:
def prepare_dataset(ds, batch_size, is_training=True):

    # Normalize images
    ds = ds.map(
        lambda x, y: (tf.cast(x, tf.float32) / 255.0, y),
        num_parallel_calls=AUTOTUNE
    )

    if is_training:
        ds = ds.shuffle(4000)
        
    ds = ds.batch(batch_size)
    
        
    return ds.prefetch(AUTOTUNE)

In [ ]:
# data_aug images
"""
if is_training:
    ds = ds.map(
        lambda x,y: (data_augmentation(x,training=True),y),
        num_parallel_calls=AUTOTUNE
    )"""

In [6]:
def get_class_distribution(dataset):
    class_counts = tf.zeros(5, dtype=tf.int32)
    for _, labels in dataset:
        class_counts += tf.cast(labels, tf.int32) # each individual sample's label in the dataset ; ( use reduce_sum if it was batched)
    return class_counts.numpy()


In [7]:
def plot_training_history(train_acc,val_acc,train_loss,val_loss):
    plt.figure(figsize=(12, 6))
    
    plt.subplot(1, 2, 1)
    plt.plot(train_acc, label='Training Accuracy')
    plt.plot(val_acc, label='Validation Accuracy')
    plt.title('Model Accuracy')
    plt.xlabel('Epoch')
    
    plt.ylabel('Accuracy')
    plt.legend()
    
    plt.subplot(1, 2, 2)
    plt.plot(train_loss, label='Training Loss')
    plt.plot(val_loss, label='Validation Loss')
    plt.title('Model Loss')
    plt.xlabel('Epoch')
    plt.ylabel('Loss')
    plt.legend()
    
    plt.show()

In [8]:
def create_model1():
       regularization_value = 0.0001    
       model = Sequential([
              InputLayer(shape=input_shape),
              Conv2D(8,(3,3),
                     strides=1,
                     #kernel_initializer='he_normal',
                     activation='relu',
                     kernel_regularizer=l2(regularization_value),
                     ),
              MaxPooling2D(),
              #Dropout(0.1),
              Conv2D(16,(3,3),
                     strides=1,
                     #kernel_initializer='he_normal',
                     activation='relu',
                     kernel_regularizer=l2(regularization_value)
                     ),
              MaxPooling2D(),
              Conv2D(8,(3,3),
                     strides=1,
                     #kernel_initializer='he_normal',
                     activation='relu',
                     kernel_regularizer=l2(regularization_value)
                     ),
              MaxPooling2D(),
              #Dropout(0.2), 
              Flatten(),
                     
              Dense(32,
                     #kernel_initializer='he_normal',
                     activation='relu',
                     kernel_regularizer=l2(regularization_value)
                     ),
              Dropout(0.4),
              Dense(5,
                     activation='softmax',
                     #kernel_initializer='glorot_uniform'
                     )
       ])
       return model

In [9]:

dataset = tf.keras.utils.image_dataset_from_directory(
    data_dir,
    image_size=(128, 128),
    label_mode='categorical',# Ensure labels are one-hot encoded
    batch_size= None,
)
# Check class names and indices
class_names = dataset.class_names
for i, class_name in enumerate(class_names):
    print(f"Class: {class_name}, Index: {i}\n")
    

In [10]:
DATASET_SIZE = tf.data.experimental.cardinality(dataset).numpy()
# Define the sizes of your splits
train_size = int(0.8 * DATASET_SIZE)
val_size = int(0.1 * DATASET_SIZE)
test_size = DATASET_SIZE - train_size - val_size

# Split the dataset
train_dataset = dataset.take(train_size)
remaining = dataset.skip(train_size)
val_dataset = remaining.take(val_size)
test_dataset = remaining.skip(val_size)

In [11]:
# Get one batch of data
for images, labels in train_dataset.take(1):  # Take 1 batch
    print("Images shape:", images.shape)
    print("Labels shape:", labels.shape)
    print("Labels:", labels.numpy())  # Convert to numpy array for easier viewing

# Check the unique values in labels to determine the format
unique_labels = tf.unique(tf.reshape(labels, [-1]))  # Flatten labels to check unique values
print("Unique label values:", unique_labels.y.numpy())


In [12]:
train_dist = get_class_distribution(train_dataset)
print("\nClass distribution in training set:")
for i, count in enumerate(train_dist):
    print(f"Class {i}: {count} samples")

# Calculate class weights
total_samples = sum(train_dist)
n_classes = len(train_dist)
class_weights = {
    i: total_samples / (n_classes * count) if count> 0 else 0
    for i, count in enumerate(train_dist)
}

print("\nClass weights:")
for class_idx, weight in class_weights.items():
    print(f"Class {class_idx}: {weight:.2f}")

In [13]:
callbacks = [
    #EarlyStopping(monitor='val_loss',patience=20,min_delta=0.00001),
    ReduceLROnPlateau(monitor="val_loss",factor=0.8, patience=5,min_delta=0.001,min_lr=5e-5),
]

In [14]:
input_shape = (128,128,3)
batch_sizes = [8,16,32]
learning_rates=[0.002,0.001,0.0005,0.0001]
Epochs = 100
results1,results2,results3 = [],[],[]

train_datasets = [prepare_dataset(train_dataset, batch_size) for batch_size in batch_sizes]
val_datasets = [prepare_dataset(val_dataset, batch_size, is_training=False) for batch_size in batch_sizes]

In [ ]:
for i, batch_size in enumerate(batch_sizes):
    train_ds = train_datasets[i]
    val_ds = val_datasets[i]
    
    for lr in learning_rates:
        tf.keras.backend.clear_session()

        model1 = create_model1()

        model1.compile(optimizer=mixed_precision.LossScaleOptimizer(Adam(learning_rate=lr)),loss=CategoricalCrossentropy(),metrics=['accuracy'])
 
        training_history1 = model1.fit(train_ds,epochs=Epochs,validation_data=val_ds,class_weight=class_weights,callbacks=callbacks,verbose=0)

        # Evaluate on validation set
        val_loss1, val_accuracy1 = model1.evaluate(val_ds, verbose=0)

        print(f"FOR Batch : {batch_size} , LR : {lr} we have \tEpochs : {len(training_history1.history['loss'])} Val Accuracy : {val_accuracy1:.4f} , Val Loss : {val_loss1:.4f}")

        results1.append({
            'batch_size': batch_size,
            'learning_rate': lr,
            'val_accuracy': val_accuracy1,
            'val_loss': val_loss1,
            'history':training_history1.history
        })
        

#### remark:
- High lr works better for big batch size
- Low lr works better for small batch size

In [20]:
best_result = min(results1, key=lambda x: x['val_loss'])

best_result_hist = best_result['history']

print("\nBest parameters: MODEL 1")
print(f"Batch Size: {best_result['batch_size']}")
print(f"Learning Rate: {best_result['learning_rate']:.4f}")
print(f"Validation Accuracy: {best_result['val_accuracy']:.4f}")
print(f"Validation Loss: {best_result['val_loss']:.4f}")
print(f"Accuracy: {best_result_hist['accuracy'][-1]:.4f}")
print(f"Loss: {best_result_hist['loss'][-1]:.4f}")

plot_training_history(best_result_hist['accuracy'],best_result_hist['val_accuracy'],best_result_hist['loss'],best_result_hist['val_loss'],)


In [21]:
# Train final model with best parameters
tf.keras.backend.clear_session()
final_train_batches = prepare_dataset(train_dataset, best_result['batch_size'])
final_val_batches = prepare_dataset(val_dataset, best_result['batch_size'], is_training=False)

final_model1 = create_model1()

final_model1.compile(optimizer=Adam(learning_rate=best_result['learning_rate']),loss='categorical_crossentropy',metrics=['accuracy'])

final_history1 = final_model1.fit(final_train_batches,epochs=150,validation_data=final_val_batches,class_weight=class_weights,callbacks=callbacks,verbose=0)

In [22]:
# plot of the training accuracy and loss
plot_training_history(final_history1.history['accuracy'],final_history1.history['val_accuracy'],final_history1.history['loss'],final_history1.history['val_loss'])

In [23]:
# Evaluate on test set
final_test_batches = prepare_dataset(test_dataset, best_result['batch_size'],is_training=False)

test_loss, test_accuracy = final_model1.evaluate(final_test_batches)
print(f"\nM1 : Final Test Accuracy: {test_accuracy:.4f},Final Test Loss: {test_loss:.4f}")


#### Old 
- Final Test Accuracy: 0.9250,
- Final Test Loss: 0.1644
---
#### after adding class weights:
- Final Test Accuracy: 0.9138,
- Final Test Loss: 0.2299

### New model one
- Final Test Accuracy: 0.8642,
- Final Test Loss: 0.3685

### another train 05/12/2024

In [ ]:
year,month,day = str(datetime.now().year),str(datetime.now().month),str(datetime.now().day)
saved_date = year+'/'+month+'/'+day
test_accuracy = str(round(test_accuracy,3))
print(test_accuracy)

0.864


In [24]:
final_model1.save(os.path.join('models',f'{saved_date+test_accuracy}.keras'))